オイラー座標での2次元のRoche モデル


In [7]:
from scipy.optimize import newton_krylov, broyden1, broyden2, newton, bisect
from scipy.linalg import lu, inv
from scipy import integrate
from numpy import cosh, zeros_like, mgrid, zeros, sin, cos, pi
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt

UL W4-method

In [8]:
#numerical Jacobian
def Jac_num(x, f, eps = 1.0e-8):
    x1    = zeros_like(x)
    ndim  = x1.shape[0]
    Jac   = np.zeros((ndim,ndim))
    for i in range(ndim):
        x1[:] = x[:]
        x1[i]+= eps
        Jac[0:ndim,i] = (f(x1) - f(x)) / eps
    
    return Jac

def W4_UL(f, x, maxitr=1000, tor = 1.0e-10, dtau = 0.5):
    p = zeros_like(x)
    ndim = p.shape[0]
    x0= zeros_like(x)
    x1= zeros_like(x)
    Jac= np.shape((ndim,ndim))    
    Jac_inv= np.shape((ndim,ndim))    
    X = np.shape((ndim,ndim))
    Y = np.shape((ndim,ndim))
    ite = 0
    x0  = x
    while(True):
        ite += 1
        if(ite>maxitr) :
            print("Not Converge")
            return x0
        Jac = Jac_num(x0, f)
        Jac_inv = sci.linalg.inv(Jac)
        #LU decomp
        P, L, U = sci.linalg.lu(Jac_inv)
        X = np.dot(P,L)
        Y = U
        x1 = x0 + dtau * np.dot(X,p) 
        p  = (1.0-2.0*dtau) * p - dtau * np.dot(Y,f(x0))  

        x0 = x1
        error = abs(f(x0)).max()
 
        if(error < tor):
            return x0


質量を与えて、極半径($r_p$)を変化させながら、Roche解を探す。Roche解の密度分布

$\rho(r,\theta) = ( \frac{-\phi_g + \phi_R + C}{K(1+N)} )^N$

となる。ここで、$\phi_g$は重力ポテンシャル$\phi_R$は遠心力ポテンシャル。
ここではそれぞれ
$\phi_g = r^2 - 10$, $\phi_R = \frac{1}{2} \Omega_0^2 (r\sin \theta)^2$

としている。極$r=r_p$で密度が0になるという境界条件から

$C = \phi_g(r_p,\theta=0)$

となる。表面形状は、密度が0になるという条件より、

$-\phi_g + \phi_R + C = 0$

より数値的に求まる。

In [32]:
#質量素片の数 = nor * nomu
#nor:r方向
#nomu:mu方向
nor = 64
nomu= 64
nom = nor * nomu

# *_E: オイラー座標
r_E   = np.zeros((nor))
mu_E  = np.zeros((nomu))
# オイラー座標的な表面
r_s   = np.zeros((nomu+1))

#enclosed mass
m_r   = np.zeros(nor+1)
#質量素片
dm    = np.zeros((nor,nomu))

# *_L: ラグランジュ座標
r_L   = np.zeros((nor+1,nomu+1))
mu_L  = np.zeros((nomu+1))
r_Lc  = np.zeros((nor,nomu))
mu_Lc = np.zeros((nor,nomu))

#rho:ρ 密度
rho_E = np.zeros((nor,nomu))
rho_L = np.zeros((nor,nomu)) 

M_mu  = np.zeros(nomu)

# p = k * rho**(1 + 1/N)
N = 1.0
K = 1.0
#M:質量 G:万有引力定数
M = 1.0
G = 1.0
#角速度Ω^2
Ome2 = 5.0e-1

dm[:] = M /float(nom)
#オイラーグリッド
dr   = (1.0 - 0.0) / float(nor-1)
r_E  = np.mgrid[0.0:1.0+dr:dr]
dmu  = (1.0 - 0.0) / float(nomu-1)
mu_E = np.mgrid[0.0:1.0+dmu:dmu]
dmu  = (1.0 - 0.0) / float(nomu)
mu_L = np.mgrid[0.0:1.0+dmu:dmu]

#適当な重力ポテンシャル
def gphi(r):
    return r**2 - 10.0
#表面を計算する式
def sur(r):
    return (-gphi(r) + 0.5 * r**2 * sin_th**2 * Ome2 + C)

#r_p:極半径 初期推量を10とする
r_p = 10.0

#r_pを変えて質量がM=1になる値を探す
for ite in range(50):
    C = gphi(r_p*r_E[nor-1])
    #それぞれのθでの表面の計算
    for j in range(nomu):
        sin_th = (1.0 - mu_E[j]**2)**0.5
        r_s[j] = newton(sur, r_p)
#        print(j, r_s[j])
    #密度の計算
    for j in range(nomu):
        for i in range(nor):
            costh = mu_E[j]
            sinth = (1.0 - mu_E[j]**2)**0.5
            r     = r_s[j]*r_E[i]
            phi_rot = 0.5 * Ome2 * (r * sinth)**2
            rho_E[i][j] = abs((-gphi(r) + phi_rot + C ) / (K*(1+N)))**(N)
        M_mu[j] = 2.0*pi*integrate.simps((r_s[j]*r_E[0:nor])**2*rho_E[0:nor,j], r_s[j]*r_E[0:nor])

#質量の計算
    M_E = 2.0*integrate.simps(M_mu[0:nomu], mu_E[0:nomu])
    #軽い場合半径を増やす
    if M_E < M:
        r_p = (r_p + r_p * (M/M_E)**(1.0/3.0))/2.0
    #重い場合は半径を減らす
    if M_E > M:
        r_p = (r_p + r_p * (M/M_E)**(1.0/3.0)) / 2.0
    if (abs(M_E - M) < 1.0e-6):break

C = gphi(r_p*r_E[nor-1])

print(r_p, "Mass on the Euler coordinate is ", M_E, "and the axis rasiot is ", r_s[nomu-1]/r_s[0])


0.978113854002 Mass on the Euler coordinate is  1.00000072418 and the axis rasiot is  0.866025403784


ここから下は、ラグランジュ座標にした時の素片の位置を、オイラー的なロッシュモデルから逆算している。
(未完成)

In [33]:
def drho(r):
    return (- K*(1+N)*rho_eq**(1.0/N) -gphi(r) + 0.5 * r**2 * sin_th**2 * Ome2 + C)

In [34]:
noeq      = 1000
drho_eq   = rho_E[0][0] / float(noeq)
r_rho_eq  = np.zeros((noeq,nomu+1))
rho_eqs   = np.zeros(noeq)
rho_eqs[0]= 0.0
for i in range(noeq-1):
    rho_eqs[i] = rho_eqs[i-1] + drho_eq
    rho_eq = rho_eqs[i]
    for j in range(nomu+1):
        sin_th = (1.0 - mu_L[j]**2)**0.5
        r_rho_eq[i][j] = newton(drho, r_p)
#    print(i,rho_eqs[i], rho_E[0,0])
i_r_L    = np.zeros(nor+1)
i_r_L    = i_r_L.astype(np.int64)
i_r_L[0] = 0

rho_eqs[noeq-1]    = rho_E[0,0]
r_rho_eq[noeq-1][:]= 0.0
m_eq = 0.0
nos  = 0

for i in range(noeq-1):
    for j in range(nomu):
        mu_c = (mu_L[j+1] + mu_L[j]) * 0.5
        dmu  = (mu_L[j+1] - mu_L[j])
        r_c  = (r_rho_eq[i][j] + r_rho_eq[i+1][j]) * 0.5
        dr   = (r_rho_eq[i][j] - r_rho_eq[i+1][j])
        rho_c= (rho_eqs[i]  + rho_eqs[i+1]) * 0.5
        m_eq = m_eq + 4.0*pi * r_c**2 * dmu * dr * rho_c
#    print(i, m_eq, dm[0,0]*nomu)
    if(m_eq > dm[0,0]*nomu):
        print(nos, i, r_rho_eq[i][0], r_rho_eq[i][nomu-1])
        m_eq = m_eq - dm[0,0]*nomu
        nos  = nos + 1
        i_r_L[nos] = i
#center
for j in range(nomu):
    mu_c = (mu_L[j+1] + mu_L[j]) * 0.5
    dmu = (mu_L[j+1] - mu_L[j])
    r_c  = (r_rho_eq[noeq-2][j]) * 0.5
    dr   = (r_rho_eq[noeq-2][j]) * 0.5
    rho_c= (rho_eqs[noeq-1]  + rho_eqs[noeq-2]) * 0.5
    m_eq = m_eq + 4.0/3.0*pi * r_c**3 * dmu * rho_c
print(noeq-1, m_eq)

for i in range(nor+1):
    for j in range(nomu+1):
        r_L[i][j] = r_rho_eq[i_r_L[i]][j]

for i in range(nor):
    for j in range(nomu):
        r_Lc[i][j] = (r_L[i+1][j] + r_L[i][j]) * 0.5
        mu_Lc[i][j]= (mu_L[j] + mu_L[j+1]    ) * 0.5
        rho_L[i][j]= (rho_eqs[i_r_L[i]]  + rho_eqs[i_r_L[i+1]]) * 0.5


0 91 1.07622158356 0.935668687786
1 130 1.05285523028 0.915353944568
2 161 1.0339053385 0.898878879724
3 187 1.01773985296 0.88482458192
4 210 1.00322250543 0.872203177816
5 231 0.989781592587 0.860517627667
6 251 0.976808791881 0.849239054927
7 270 0.964322995694 0.83838388466
8 287 0.953012844963 0.828550822348
9 304 0.941566845903 0.818599653291
10 320 0.930665556041 0.809122055243
11 335 0.920328331822 0.800134856725
12 350 0.90987367228 0.791045559758
13 365 0.899297481739 0.781850603555
14 379 0.889312898293 0.773169991465
15 392 0.879940060878 0.765021232306
16 406 0.869733281614 0.756147442832
17 419 0.860147115797 0.747813215635
18 432 0.850452903152 0.739385052362
19 444 0.841405270605 0.731519026813
20 457 0.831492579082 0.72290091767
21 469 0.822236339498 0.714853528844
22 481 0.812874705544 0.706714509993
23 493 0.803403992918 0.698480657977
24 505 0.793820297523 0.690148578569
25 516 0.784932457564 0.68242147694
26 528 0.775120411201 0.673890868852
27 539 0.766015633026 0

In [35]:
file1   = open("rho_E.dat", 'w')
file2   = open("rho_L.dat", "w")
file3   = open("r_L.dat", "w")

for j in range(0,nomu):
    for i in range(0, nor):
        cos_th = mu_E[j]
        sin_th = (1.0 - mu_E[j]**2)**0.5
        r     = r_s[j]*r_E[i]
        out1 = str(r*sin_th) + " " + str(r*cos_th) + " " + str(rho_E[i][j]) + "\n"
        file1.write(out1)
    file1.write("\n")
file1.close()
for j in range(0,nomu):
    for i in range(0, nor):
        cos_th = mu_Lc[i][j]
        sin_th = (1.0 - cos_th**2)**0.5
        r     = r_Lc[i][j]
#        print(i,j, r_Lc[i][j], mu_Lc[i][j], rho_L[i][j])
        out2 = str(r*sin_th) + " " + str(r*cos_th) + " " + str(rho_L[i][j]) + "\n"
        file2.write(out2)
    file2.write("\n")
file2.close()

for i in range(0,nomu):
    for j in range(0, nor):
        cos_th = mu_E[j]
        sin_th = (1.0 - cos_th**2)**0.5
        r     = r_L[i][j]
#        print(i,j, r_Lc[i][j], mu_Lc[i][j], rho_L[i][j])
        out3 = str(r*sin_th) + " " + str(r*cos_th) + " " + "1.0" + "\n"
        file3.write(out3)
    file3.write("\n")
file3.close()

ロッシュ解をどのようにラグランジュ座標の素片の位置にするかのモデル。
Roce_2D, Roche_2D0, Roche_2D00 など

素片の座標を入れると、オイラー的な密度とラグランジュ座標から求めた密度の差を返す関数

In [36]:

def Roche_2D(r21):
    # r11 r12 r21 r22
    r_c   = (r11+r12+r21+r22) / 4.0 * r_p
    dr    = (r22 + r21 - r11 - r12) / 2.0 * r_p
    mu_c  = 0.5*(mu1 + mu2) 
    dmu   = (mu2 - mu1)
    sinth_c = (1.0 - mu_c**2)**0.5
    phi_rot = 0.5 * Ome2 * (r_c * sinth_c)**2

    rho_e = ((-gphi(r_c) + phi_rot + C ) / (K*(1+N)))**N
    dV    = 4.0 * pi * r_c**2 * dr * dmu
    rho_l = d_m / dV
#    print(r2*r_s,rho_e, rho_l, d_m, dV)
    return rho_e - rho_l

def Roche_2D0(r22):
    # r11 = r12 = 0, r21 r22
    r_c   = (r21+r22) / 4.0 * r_p
    dr    = (r22 + r21 - r11 - r12) / 2.0 * r_p
    mu_c  = 0.5*(mu1 + mu2) 
    dmu   = (mu2 - mu1)
    sinth_c = (1.0 - mu_c**2)**0.5
    phi_rot = 0.5 * Ome2 * (r_c * sinth_c)**2

    rho_e = ((-gphi(r_c) + phi_rot + C ) / (K*(1+N)))**N
    dV    = 4.0 /3.0 * pi * r_c**3 * dmu
    rho_l = d_m / dV
#    print(r2*r_s,rho_e, rho_l, d_m, dV)
    return rho_e - rho_l

def Roche_2D00(r2):
    r_c   = 0.5 * r2 * r_p
    dr    =       r2 * r_p
    rho_e = ((-gphi(r_c) + C ) / (K*(1+N)))**N
    dV    = 4.0 / 3.0 * pi * (r_p*r2)**3
    rho_l = d_m / dV
#    print(rho_e, rho_l, d_m, dV)
    return rho_e - rho_l

def Roche_2Di0(r2):
    r_c   = 0.5 * (r2 + r1) * r_p
    dr    =       (r2 - r1) * r_p
    rho_e = ((-gphi(r_c) + C ) / (K*(1+N)))**N
    dV    = 4.0 * pi * (r_c)**2 * dr
    rho_l = d_m / dV
#    print(rho_e, rho_l, d_m, dV)
    return rho_e - rho_l

In [54]:
#on the rotational axis

d_m= dm[0,0] * (nomu)
r_L[0][:]   = 0.0
r_L[1][:]   = newton(Roche_2D00,0.04)
dV    = 4.0 / 3.0 * pi * (r_L[1][0]*r_p)**3
rho_L[0][:] = d_m / dV

print(0,r_p*r_L[1][nomu])
for i in range(1,nor):
    d_m= dm[0][0] * nomu
    r1 = r_L[i][nomu]
    r_L[i+1][nomu] = newton(Roche_2Di0,r_L[i][nomu]+1.0e-3)
    print(i+1, nomu, r_L[i+1][nomu])
print(nor, nomu, r_L[nor][nomu])

0 0.198990920797
2 64 0.258456184407
3 64 0.297424073428
4 64 0.328833796774
5 64 0.355692380815
6 64 0.379462747398
7 64 0.400981328321
8 64 0.42077701861
9 64 0.439208350807
10 64 0.456531170833
11 64 0.472935434328
12 64 0.488566727479
13 64 0.503539592618
14 64 0.517946167904
15 64 0.531862006237
16 64 0.54535012267
17 64 0.558463889114
18 64 0.571249156275
19 64 0.583745844251
20 64 0.595989160029
21 64 0.608010548469
22 64 0.61983845051
23 64 0.631498920835
24 64 0.643016143178
25 64 0.654412871916
26 64 0.665710822393
27 64 0.6769310284
28 64 0.688094182988
29 64 0.699220977927
30 64 0.710332457667
31 64 0.721450405705
32 64 0.732597785195
33 64 0.743799262249
34 64 0.755081850961
35 64 0.766475736026
36 64 0.778015356072
37 64 0.789740875842
38 64 0.801700252286
39 64 0.813952236406
40 64 0.826570907909
41 64 0.839652844873
42 64 0.853329104564
43 64 0.867786691296
44 64 0.883310730576
45 64 0.900378680553
46 64 0.919916338913
47 64 0.944287667908
48 64 0.990664271281


RuntimeError: Failed to converge after 50 iterations, value is 0.926465978073

In [55]:
mu1 = 0.0
mu2 = 0.0
    
r11 = 0.0
r12 = 0.0
r22 = r_L[1][0]

#r_L[1][1] = newton(Roche_2D,r21*1.01)
#print(1, r_p*r_L[1][0], r_p*r_L[1][1])
#print(r_Lc[0][:])
for j in range(nomu-1, -1, -1):
#    print(1, j, r_p*r_L[1][j+1],r_p*r_L[1][j])
    for i in range(2,nor+1):
        d_m= dm[0,0]
        mu1 = mu_L[i][j]
        mu2 = mu_L[i][j+1]

        r11 = r_L[i-1][j]
        r12 = r_L[i-1][j+1]
        r22 = r_L[i][j+1]
        r_L[i][j] = newton(Roche_2D,r22+1.0e-3)
        r21 = r_L[i][j]
        r_c   = (r11+r12+r21+r22) / 4.0 * r_p
        dr    = (r22 + r21 - r11 - r12) / 2.0 * r_p
        mu_c  = 0.5*(mu1 + mu2) 
        dmu   = (mu2 - mu1)
        dV    = 4.0 * pi * r_c**2 * dr * dmu
        r_Lc[i-1][j]  = r_c
        mu_Lc[i-1][j]   = mu_c
        mu_Lc[0][j]   = mu_c

        rho_L[i-1][j] = d_m / dV
  
    print(i,j,r_L[nor][j])

#print(nor-1, r_L[nor][0])

IndexError: invalid index to scalar variable.

In [40]:
file1   = open("rho_E.dat", 'w')
file1_r = open("rho_E_r.dat", 'w')
file2   = open("rho_L.dat", "w")
file2_r = open("rho_L_r.dat", "w")

for j in range(0,nomu):
    for i in range(0, nor):
        cos_th = mu_E[j]
        sin_th = (1.0 - mu_E[j]**2)**0.5
        r     = r_s[j]*r_E[i]
        out1 = str(r*sin_th) + " " + str(r*cos_th) + " " + str(rho_E[i][j]) + "\n"
        file1.write(out1)
    file1.write("\n")
file1.close()
for j in range(0,nomu):
    for i in range(0, nor):
        cos_th = mu_Lc[i][j]
        sin_th = (1.0 - cos_th**2)**0.5
        r     = r_p*r_Lc[i][j]
#        print(i,j, r_Lc[i][j], mu_Lc[i][j], rho_L[i][j])
        out2 = str(r*sin_th) + " " + str(r*cos_th) + " " + str(rho_L[i][j]) + "\n"
        file2.write(out2)
    file2.write("\n")
file2.close()